# Импорты

In [39]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
from datasets import load_dataset

# Чтение датасетов

In [46]:
df_1 = load_dataset("Kasymkhan/RussianFinancialNews")

df_1

Using the latest cached version of the dataset since Kasymkhan/RussianFinancialNews couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /Users/ruslan/.cache/huggingface/datasets/Kasymkhan___russian_financial_news/default/0.0.0/f83da5c05c82c9086806cf90ded4155052f9605c (last modified on Sat Jan 25 13:02:34 2025).


DatasetDict({
    train: Dataset({
        features: ['title', 'body', 'date', 'time', 'tags', 'source', '__index_level_0__'],
        num_rows: 73564
    })
    test: Dataset({
        features: ['title', 'body', 'date', 'time', 'tags', 'source', '__index_level_0__'],
        num_rows: 18391
    })
})

In [47]:
df_2 = pd.read_csv("./data/data.tsv", sep="\t", usecols=["title", "summary", "published", "tickers"])

df_2.head()

,title,summary,published,tickers
0,Электромобильный стартап Arrival экс-главы Yot...,"Британский электромобильный стартап Arrival, к...","Thu, 12 May 2022 05:10:01 +0300",['ARVL']
1,Экс-глава НМТП рассказал о «напряженных отноше...,Экс-председатель совета директоров Новороссийс...,"Fri, 20 May 2022 19:13:18 +0300",['NMTP']
2,Шрёдер отклонил предложение войти в совет дире...,Его кандидатуру выдвинули в начале февраля. Ка...,"Tue, 24 May 2022 22:12:05 +0300",['GAZP']
3,Шельф берут в разработку // Генподрядчиком «Га...,"Как стало известно “Ъ”, «Аврора» Андрея Патруш...","Fri, 29 Jul 2022 00:28:00 +0300",['GAZP']
4,"Чистый убыток ""Юнипро"" в 1 полугодии 2022 года...","Чистый убыток ""Юнипро"" в 1 полугодии 2022 года...","Thu, 28 Jul 2022 12:43:00 +0300",['UPRO']


# Предобработка датасетов

### Датасет из HF

In [48]:
df_1 = pd.concat([df_1["train"].to_pandas(), df_1["test"].to_pandas()], axis=0)
df_1 = df_1[["body", "date", "tags"]]

df_1.head()

,body,date,tags
0,Российские отели «всё включено» обяжут предлаг...,2023-03-10,no tags
1,""" Самолет (SMLT) в 4кв2023 собирается побить р...",2023-10-26,"['аналитика', 'факты']"
2,"""Высокие цены на нефть, похоже, надолго. Мир в...",2022-08-29,"['ROSN', 'LKOH']"
3,ЦБ определил порядок продажи заблокированных и...,2023-12-11,no tags
4,Чем привлекателен выпуск Газпром Капитала,2024-01-12 00:00:00,['Газпром капитал ЗО28-1-Е RU000A105BY1']


In [51]:
df_1["date"] = pd.to_datetime(df_1["date"], format="mixed")

In [ ]:
df_1["tags"] = df_1["tags"].map(lambda x: None if x == "no tags" else x)

df_1.isna().sum()

### Датасет из GitHub

In [19]:
df_2["published"] = pd.to_datetime(df_2["published"], format="mixed")

/var/folders/7h/x164jwhx1ld31rc774tg1n0m0000gn/T/ipykernel_1955/782301921.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df_2["published"] = pd.to_datetime(df_2["published"], format="mixed")


In [59]:
df_2.isna().sum()

title         0
summary      12
published     0
tickers       0
dtype: int64

In [64]:
df_2 = df_2.loc[df_2['summary'].notna(), ['title', 'published', 'summary', 'tickers']]

# Объединение датасетов

In [77]:
### переименовываем столбцы в df_1

df_1 = df_1.rename(columns={
    "body": "news",
    "date": "date",
    "tags": "tags"
})

In [78]:
df_2 = df_2.rename(columns={
    "title": "title",
    "published": "date",
    "summary": "news",
    "tickers": "tags"
})

In [83]:
pd.concat([df_1, df_2], axis=0).to_csv('./data/merged_news_dataset.csv', index=False)